In [1]:
import pandas as pd
import re
import numpy as np
from functools import reduce
import pandas as pd
import seaborn as sns
from igel import Igel

In [2]:
def clean_dataset_int(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df[indices_to_keep].astype(np.float64)

def clean_final_dataset(df):
    df = df.drop(['category','eventID','eventType','eventID'],axis =1)
    return df

def merge_datasets(df):
    dfs = [dataset_labelled, df]
    df_final = reduce(lambda left,right: pd.merge(left,right,on='tweet_id'), dfs)
    return df_final

def clear_res(res):
    res['tweet_id'] = pd.to_numeric(res['tweet_id'].apply(lambda x: re.sub(',', '.', str(x))))
    del res['numb_of_mentions']
    del res['numb_of_urls']
    del res['numb_of_hashtags']
    del res['numb_of_personal_pronouns']
    del res['numb_of_present_tenses']
    del res['positive_sentiment']
    del res['negative_sentiment']
    del res['numb_of_questions']
    del res['tweet_length']
    del res['userFollowersCount']
    del res['userFriendsCount']
    del res['user_list_count']
    del res['dict_precision']
    del res['dict_recall']
    del res['dict_f_measure']
    return res

def clear_vec(vec):
    del vec['positive_sentiment']
    del vec['negative_sentiment']
    del vec['numb_of_questions']
    del vec['tweet_length']
    del vec['userFollowersCount']
    del vec['userFriendsCount']
    del vec['user_list_count']
    del vec['dict_precision']
    del vec['dict_recall']
    del vec['dict_f_measure']
    return vec

def trim(df):
    df['timestamp'] = df['timestamp'].str[1:]
    df['timestamp'] = df['timestamp'].str[:-1]
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df
    
    
def clean_brackets(df):
    df['tweet_id'] = df['tweet_id'].str[:-1]
    df['positive_sentiment'] = df['positive_sentiment'].str[:-1]
    df['negative_sentiment'] = df['negative_sentiment'].str[:-1]
    df['numb_of_questions'] = df['numb_of_questions'].str[:-1]
    df['tweet_length'] = df['tweet_length'].str[:-1]
    df['userFollowersCount'] = df['userFollowersCount'].str[:-1]
    df['userFriendsCount'] = df['userFriendsCount'].str[:-1]
    df['user_list_count'] = df['user_list_count'].str[:-1]
    df['dict_precision'] = df['dict_precision'].str[:-1]
    df['dict_recall'] = df['dict_recall'].str[:-1]
    df['dict_f_measure'] = df['dict_f_measure'].str[:-1]
    df['numb_of_media'] = df['numb_of_media'].str[:-1]
    #df.tweet_id.map(lambda x: '{:.0f}'.format(x))
    #df.tweet_id = df.tweet_id.map(lambda x: '{:.0f}'.format(x))
    df['tweet_id'] = pd.to_numeric(df['tweet_id'].apply(lambda x: re.sub(',', '.', str(x))))
    df['tweet_id'].describe()
    return df

def clean_float(df):
    # Obj -> Str -> float   
    pd.set_option('display.float_format', '{:.2f}'.format)
    for col in df.columns.values:
        print(col)
        df[col] = df[col].astype(str).astype(float)
    float_col = df.select_dtypes(include=['float64'])
    for col in float_col.columns.values:
        df[col] = df[col].astype('int64')
    display(df.dtypes)
    return df

def clean_initial_feature_vector(df):
    df = trim(df)
    df = clean_brackets(df)
    return df

In [3]:
# Set the column names
feature_vector_keys = ["timestamp", "tweet_id", "positive_sentiment", "negative_sentiment", \
                       "numb_of_mentions", "numb_of_urls", "numb_of_media","numb_of_hashtags", \
                       "numb_of_personal_pronouns", "numb_of_present_tenses", "numb_of_past_tenses", \
                       "sent_from_web", "numb_of_weird_chars", "numb_of_questions", "numb_of_emoticons", \
                       "numb_of_swearing_words", "numb_of_slang_words", "numb_of_intensifiers", \
                       "tweet_length", "userFollowersCount","userFriendsCount", "user_numb_of_tweets",\
                       "user_list_count", "tfidf_fire", "dict_precision", "dict_recall", "dict_f_measure"
                       ]

# The Topic IDs that we'll map
eventidTopicidMap =	{
  "athensEarthquake2020": "TRECIS-CTIT-H-Test-035",
  "baltimoreFlashFlood2020": "TRECIS-CTIT-H-Test-036",
  "brooklynBlockPartyShooting2020": "TRECIS-CTIT-H-Test-037",
  "daytonOhioShooting2020": "TRECIS-CTIT-H-Test-038",
  "elPasoWalmartShooting2020": "TRECIS-CTIT-H-Test-039",
  "gilroygarlicShooting2020": "TRECIS-CTIT-H-Test-040",
  "hurricaneBarry2020": "TRECIS-CTIT-H-Test-041",
  "indonesiaEarthquake2020": "TRECIS-CTIT-H-Test-042",
  "keralaFloods2020": "TRECIS-CTIT-H-Test-043",
  "myanmarFloods2020": "TRECIS-CTIT-H-Test-044",
  "papuaNewguineaEarthquake2020": "TRECIS-CTIT-H-Test-045",
  "siberianWildfires2020": "TRECIS-CTIT-H-Test-046",
  "typhoonKrosa2020": "TRECIS-CTIT-H-Test-047",
  "typhoonLekima2020": "TRECIS-CTIT-H-Test-048",
  "whaleyBridgeCollapse2020": "TRECIS-CTIT-H-Test-049"
}

priority_scorer = {
    '1.0' : 'Critical',
    '0.9' : 'Critical',
    '0.8' : 'High',
    '0.7' : 'High',
    '0.6' : 'High',
    '0.5' : 'Medium',
    '0.4' : 'Medium',
    '0.2' : 'Low',
    '0.3' : 'Medium',
    '0.0' : 0.0,
}

categoriser = {
    0 : 'Advice',
    1.0 : 'CleanUp',
    2 : 'ContextualInformation',
    3 : 'Discussion',
    4 : 'Donations',
    5 : 'EmergingThreats',
    6 : 'Factoid',
    7 : 'FirstPartyObservation',
    8 : 'GoodsServices',
    9 : 'Hashtags',
    10 : 'InformationWanted',
    11 : 'Irrelevant',
    12.0 : 'Location',
    13 : 'MovePeople',
    14 : 'MultimediaShare',
    15 : 'NewSubEvent',
    16 : 'News',
    17 : 'Official',
    18 : 'OriginalEvent',
    19 : 'SearchAndRescue',
    20 : 'Sentiment',
    21 : 'ServiceAvailable',
    22 : 'ThirdPartyObservation',
    23 : 'Volunteer',
    24 : 'Weather',
}



In [4]:
# Currently loading the feature vector from a .txt file exported from Play
feature_vector_input = pd.read_csv("../../../0-data/processed/all_new.txt", sep=" ", header=None)

# Set the column names
feature_vector_input.columns = feature_vector_keys

# Group by tweet
feature_vector_input = feature_vector_input.groupby(['tweet_id']).agg('first')

# Reset the index
feature_vector_input.reset_index(level=0, inplace=True)

# Clean & drop empty rows
feature_vector_input = clean_initial_feature_vector(feature_vector_input)

feature_vector_input

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1.128285e+18,1.557839196E12,3.0,60.0,"0.0,","0.0,",2.0,"0.0,","0.0,","0.0,",...,"0.0,",219.0,4416.0,4860.0,"0.0,",29.0,"0.0,",0.0,0.0,0.0
1,1.128286e+18,1.557839239E12,2.0,66.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",139.0,2420.0,490.0,"0.0,",11.0,"0.0,",0.0,0.0,0.0
2,1.128286e+18,1.557839245E12,3.0,60.0,"0.0,","0.0,",2.0,"0.0,","0.0,","0.0,",...,"0.0,",220.0,232.0,841.0,"0.0,",1.0,"0.0,",0.0,0.0,0.0
3,1.128286e+18,1.557839261E12,3.0,65.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",114.0,4680.0,4.0,"0.0,",107.0,"0.0,",0.0,0.0,0.0
4,1.128286e+18,1.557839266E12,3.0,65.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",114.0,3179.0,3.0,"0.0,",64.0,"0.0,",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9702,1.162005e+18,1.5658785E12,5.0,57.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",113.0,5543.0,169.0,"0.0,",200.0,"0.0,",0.0,0.0,0.0
9703,1.162005e+18,1.565878597E12,8.0,51.0,"0.0,","0.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",98.0,83.0,328.0,"0.0,",0.0,"0.0,",0.0,0.0,0.0
9704,1.162006e+18,1.565878707E12,15.0,14.0,"0.0,","0.0,",0.0,"0.0,","0.0,","0.0,",...,"0.0,",124.0,65.0,98.0,"0.0,",0.0,"0.0,",0.0,0.0,0.0
9705,1.162006e+18,1.56587876E12,37.0,22.0,"0.0,","1.0,",1.0,"0.0,","0.0,","0.0,",...,"0.0,",239.0,1129.0,1049.0,"0.0,",174.0,"0.0,",0.0,0.0,0.0


In [5]:
# Create a numeric version for our model
df_int = clean_dataset_int(feature_vector_input)
df_int

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1.128285e+18,1.557839e+11,30.0,600.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,2190.0,44160.0,48600.0,0.0,290.0,0.0,0.0,0.0,0.0
1,1.128286e+18,1.557839e+11,20.0,660.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1390.0,24200.0,4900.0,0.0,110.0,0.0,0.0,0.0,0.0
2,1.128286e+18,1.557839e+11,30.0,600.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,2200.0,2320.0,8410.0,0.0,10.0,0.0,0.0,0.0,0.0
3,1.128286e+18,1.557839e+11,30.0,650.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1140.0,46800.0,40.0,0.0,1070.0,0.0,0.0,0.0,0.0
4,1.128286e+18,1.557839e+11,30.0,650.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1140.0,31790.0,30.0,0.0,640.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9702,1.162005e+18,1.565879e+09,50.0,570.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1130.0,55430.0,1690.0,0.0,2000.0,0.0,0.0,0.0,0.0
9703,1.162005e+18,1.565879e+11,80.0,510.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,980.0,830.0,3280.0,0.0,0.0,0.0,0.0,0.0,0.0
9704,1.162006e+18,1.565879e+11,150.0,140.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1240.0,650.0,980.0,0.0,0.0,0.0,0.0,0.0,0.0
9705,1.162006e+18,1.565879e+10,370.0,220.0,0.0,10.0,10.0,0.0,0.0,0.0,...,0.0,2390.0,11290.0,10490.0,0.0,1740.0,0.0,0.0,0.0,0.0


In [6]:
# Load the provided Labels - These are generated in 0_Labels.ipynb
dataset_labelled = pd.read_csv("../3-csv/dataset_labelled_tweets.csv")
dataset_labelled

,tweet_id,priority,category,postCategories_x,eventID,eventType
0,1128285482784366592,0.75,Location,12,papuaNewguineaEarthquake2020,Unknown
1,1128285482784366592,0.75,Factoid,12,papuaNewguineaEarthquake2020,Unknown
2,1128285665186197504,0.25,Location,12,papuaNewguineaEarthquake2020,Unknown
3,1128285665186197504,0.25,Factoid,12,papuaNewguineaEarthquake2020,Unknown
4,1128285665186197504,0.25,Hashtags,12,papuaNewguineaEarthquake2020,Unknown
...,...,...,...,...,...,...
15264,1162006062867918848,0.25,Irrelevant,16,typhoonKrosa2020,Unknown
15265,1162006062867918848,0.25,Location,16,typhoonKrosa2020,Unknown
15266,1162006062867918848,0.25,MultimediaShare,16,typhoonKrosa2020,Unknown
15267,1162006062867918848,0.25,Hashtags,16,typhoonKrosa2020,Unknown


In [7]:
# Merges the input feature vector with the labels
merged_df = merge_datasets(df_int)

# Export as Igel takes a .csv input
merged_df.to_csv("../3-csv/merged_df.csv", index=False)
merged_df

,tweet_id,priority,category,postCategories_x,eventID,eventType,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1128285482784366592,0.75,Location,12,papuaNewguineaEarthquake2020,Unknown,1.557839e+11,30.0,600.0,0.0,...,0.0,2190.0,44160.0,48600.0,0.0,290.0,0.0,0.0,0.0,0.0
1,1128285482784366592,0.75,Factoid,12,papuaNewguineaEarthquake2020,Unknown,1.557839e+11,30.0,600.0,0.0,...,0.0,2190.0,44160.0,48600.0,0.0,290.0,0.0,0.0,0.0,0.0
2,1128285757624311808,0.25,Location,12,papuaNewguineaEarthquake2020,Unknown,1.557839e+11,30.0,650.0,0.0,...,0.0,1140.0,46800.0,40.0,0.0,1070.0,0.0,0.0,0.0,0.0
3,1128285757624311808,0.25,Factoid,12,papuaNewguineaEarthquake2020,Unknown,1.557839e+11,30.0,650.0,0.0,...,0.0,1140.0,46800.0,40.0,0.0,1070.0,0.0,0.0,0.0,0.0
4,1128286351760265216,0.25,Location,12,papuaNewguineaEarthquake2020,Unknown,1.557839e+11,30.0,540.0,0.0,...,0.0,2350.0,130.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12765,1162006062867918848,0.25,Irrelevant,16,typhoonKrosa2020,Unknown,1.565879e+11,50.0,610.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12766,1162006062867918848,0.25,Location,16,typhoonKrosa2020,Unknown,1.565879e+11,50.0,610.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12767,1162006062867918848,0.25,MultimediaShare,16,typhoonKrosa2020,Unknown,1.565879e+11,50.0,610.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12768,1162006062867918848,0.25,Hashtags,16,typhoonKrosa2020,Unknown,1.565879e+11,50.0,610.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0


In [8]:
# Remove non-numeric values 
merged_df_int = clean_final_dataset(merged_df)

# Export as Igel takes a .csv input
merged_df_int.to_csv("../3-csv/merged_df_int.csv", index=False)
merged_df_int

,tweet_id,priority,postCategories_x,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1128285482784366592,0.75,12,1.557839e+11,30.0,600.0,0.0,0.0,20.0,0.0,...,0.0,2190.0,44160.0,48600.0,0.0,290.0,0.0,0.0,0.0,0.0
1,1128285482784366592,0.75,12,1.557839e+11,30.0,600.0,0.0,0.0,20.0,0.0,...,0.0,2190.0,44160.0,48600.0,0.0,290.0,0.0,0.0,0.0,0.0
2,1128285757624311808,0.25,12,1.557839e+11,30.0,650.0,0.0,0.0,10.0,0.0,...,0.0,1140.0,46800.0,40.0,0.0,1070.0,0.0,0.0,0.0,0.0
3,1128285757624311808,0.25,12,1.557839e+11,30.0,650.0,0.0,0.0,10.0,0.0,...,0.0,1140.0,46800.0,40.0,0.0,1070.0,0.0,0.0,0.0,0.0
4,1128286351760265216,0.25,12,1.557839e+11,30.0,540.0,0.0,0.0,10.0,0.0,...,0.0,2350.0,130.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12765,1162006062867918848,0.25,16,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12766,1162006062867918848,0.25,16,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12767,1162006062867918848,0.25,16,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12768,1162006062867918848,0.25,16,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0


In [9]:
# Remove the priority for our prediction
merged_df_wo_priority = merged_df_int

del merged_df_wo_priority['priority']
del merged_df_wo_priority['postCategories_x']

merged_df_wo_priority.to_csv("../3-csv/merged_df_wo_priority.csv", index=False)
merged_df_wo_priority

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1128285482784366592,1.557839e+11,30.0,600.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,2190.0,44160.0,48600.0,0.0,290.0,0.0,0.0,0.0,0.0
1,1128285482784366592,1.557839e+11,30.0,600.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,2190.0,44160.0,48600.0,0.0,290.0,0.0,0.0,0.0,0.0
2,1128285757624311808,1.557839e+11,30.0,650.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1140.0,46800.0,40.0,0.0,1070.0,0.0,0.0,0.0,0.0
3,1128285757624311808,1.557839e+11,30.0,650.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,1140.0,46800.0,40.0,0.0,1070.0,0.0,0.0,0.0,0.0
4,1128286351760265216,1.557839e+11,30.0,540.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,2350.0,130.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12765,1162006062867918848,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12766,1162006062867918848,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12767,1162006062867918848,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0
12768,1162006062867918848,1.565879e+11,50.0,610.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,1170.0,33680.0,1810.0,0.0,15480.0,0.0,0.0,0.0,0.0


In [10]:
# Select the options for ML: [cluster, hyper, iris, regres, forest, multi]
#Fit
algo_fit = 'yaml/multi.yaml'

# Evaluate
algo_eval = 'yaml/hyper.yaml'

# Predict
algo_predict = 'yaml/hyper.yaml'   

In [11]:
# Fit
params = {
        'cmd': 'fit',    # provide the command you want to use. whether fit, evaluate or predict
        'data_path': "../3-csv/merged_df_int.csv",
        'yaml_path': algo_fit
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'fit', 'data_path': '../3-csv/merged_df_int.csv', 'yaml_path': 'yaml/multi.yaml'}
INFO - Executing command: fit ...
INFO - reading data from ../3-csv/merged_df_int.csv
INFO - You passed the configurations as a yaml file.
INFO - your chosen configuration: {'dataset': {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default'}}, 'model': {'type': 'regression', 'algorithm': 'RandomForest'}, 'target': ['priority', 'postCategories_x']}
INFO - dataset_props: {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default'}} 
model_props: {'type': 'regression', 'algorithm': 'RandomForest'} 
 target: ['priority', 'postCategories_x'] 

INFO - dataset shape: (12770, 29)
INFO - dataset attributes: ['tweet_id', 'priority', 'postCategories_x', 'timestamp', 'positive_sentiment', 'negative_sentiment', 'numb_of_mentions', 'numb_of_urls', 'numb_of_media', 'numb_of_hashtags', 'numb_of_personal_pronouns', 'numb_of_present_tenses', 'numb_of_past_tenses', 'sent_

In [12]:
# Evaluate
params = {
        'cmd': 'evaluate',    # provide the command you want to use. whether fit, evaluate or predict
        'data_path': "../3-csv/merged_df_int.csv",
        'yaml_path': algo_eval
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'evaluate', 'data_path': '../3-csv/merged_df_int.csv', 'yaml_path': 'yaml/hyper.yaml'}
INFO - Executing command: evaluate ...
INFO - reading data from ../3-csv/merged_df_int.csv
INFO - path of the pre-fitted model => /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav
INFO - result path: /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results 
INFO - loading model form /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav 
INFO - dataset shape: (12770, 29)
INFO - dataset attributes: ['tweet_id', 'priority', 'postCategories_x', 'timestamp', 'positive_sentiment', 'negative_sentiment', 'numb_of_mentions', 'numb_of_urls', 'numb_of_media', 'numb_of_hashtags', 'numb_of_personal_pronouns', 'numb_of_present_tenses', 'numb_of_past_tenses', 'sent_from_web', 'numb_of_weird_chars', 'numb_of_questions', 'numb_of_emoticons', 'numb_of_swearing_words', 'numb_of_slang_words', 'numb_

In [13]:
# Predict
params = {
        'cmd': 'predict',    # provide the command you want to use. whether fit, evaluate or predict
        'data_path': "../3-csv/merged_df_wo_priority.csv",
        'yaml_path': algo_predict
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'predict', 'data_path': '../3-csv/merged_df_wo_priority.csv', 'yaml_path': 'yaml/hyper.yaml'}
INFO - Executing command: predict ...
INFO - reading data from ../3-csv/merged_df_wo_priority.csv
INFO - path of the pre-fitted model => /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav
INFO - loading from /Users/mark/Documents/GitHub/HelpMe/1-src/2-python/2-notebooks/model_results/model.sav
INFO - dataset shape: (12770, 27)
INFO - dataset attributes: ['tweet_id', 'timestamp', 'positive_sentiment', 'negative_sentiment', 'numb_of_mentions', 'numb_of_urls', 'numb_of_media', 'numb_of_hashtags', 'numb_of_personal_pronouns', 'numb_of_present_tenses', 'numb_of_past_tenses', 'sent_from_web', 'numb_of_weird_chars', 'numb_of_questions', 'numb_of_emoticons', 'numb_of_swearing_words', 'numb_of_slang_words', 'numb_of_intensifiers', 'tweet_length', 'userFollowersCount', 'userFriendsCount', 'user_numb_of_tweets', 'user_list_count', 'tfid

In [14]:
# Load the predictions
predictions = pd.read_csv("model_results/predictions.csv")

predictions = predictions.sort_values(by=['priority'])
predictions

,priority,postCategories_x
12769,0.25,16.00
11164,0.25,11.00
11165,0.25,24.00
4845,0.25,24.00
4844,0.25,24.00
...,...,...
1333,1.00,19.00
5480,1.00,18.00
3667,1.00,16.02
3665,1.00,16.02


In [15]:
# Merge the new predictions back onto dataframe with the missing columns
df = merged_df_wo_priority.merge(predictions, left_index=True, right_index=True)
cleaned_wo = df.filter(['tweet_id','priority','postCategories_x'], axis=1)
cleaned_wo
#merged_predictions = clear_res(df)
#merged_predictions

,tweet_id,priority,postCategories_x
0,1128285482784366592,0.67000,12.04
1,1128285482784366592,0.67000,12.04
2,1128285757624311808,0.28625,12.02
3,1128285757624311808,0.28625,12.02
4,1128286351760265216,0.26000,12.04
...,...,...,...
12765,1162006062867918848,0.25000,16.00
12766,1162006062867918848,0.25000,16.00
12767,1162006062867918848,0.25000,16.00
12768,1162006062867918848,0.25000,16.00


In [16]:
# Merge the predictions back in with our labelled data
df_merged = pd.merge(cleaned_wo, merged_df, left_on = 'tweet_id', right_on = 'tweet_id', how = 'inner')

df_merged.drop_duplicates(inplace=True)

df = df_merged.groupby('tweet_id')['category'].apply(list)

df = df.to_frame()

df_merg = pd.merge(df_merged, df, left_on = 'tweet_id', right_on = 'tweet_id', how = 'inner')


df_merg

cleaned_merg = df_merg.filter(['tweet_id','eventID','priority_x','priority_y','category_x','category_y', 'postCategories_x_x','postCategories_x_y'], axis=1)
cleaned_merg


,tweet_id,eventID,priority_x,priority_y,category_x,category_y,postCategories_x_x,postCategories_x_y
0,1128285482784366592,papuaNewguineaEarthquake2020,0.67000,0.75,Location,"[Location, Factoid]",12.04,12
1,1128285482784366592,papuaNewguineaEarthquake2020,0.67000,0.75,Factoid,"[Location, Factoid]",12.04,12
2,1128285757624311808,papuaNewguineaEarthquake2020,0.28625,0.25,Location,"[Location, Factoid]",12.02,12
3,1128285757624311808,papuaNewguineaEarthquake2020,0.28625,0.25,Factoid,"[Location, Factoid]",12.02,12
4,1128286351760265216,papuaNewguineaEarthquake2020,0.26000,0.25,Location,"[Location, Factoid, Hashtags]",12.04,12
...,...,...,...,...,...,...,...,...
9890,1162006062867918848,typhoonKrosa2020,0.25000,0.25,Irrelevant,"[Irrelevant, Location, MultimediaShare, Hashta...",16.00,16
9891,1162006062867918848,typhoonKrosa2020,0.25000,0.25,Location,"[Irrelevant, Location, MultimediaShare, Hashta...",16.00,16
9892,1162006062867918848,typhoonKrosa2020,0.25000,0.25,MultimediaShare,"[Irrelevant, Location, MultimediaShare, Hashta...",16.00,16
9893,1162006062867918848,typhoonKrosa2020,0.25000,0.25,Hashtags,"[Irrelevant, Location, MultimediaShare, Hashta...",16.00,16


In [17]:
# write to .run file
with open("marks2.run" , "w") as out_file:
    for row in df_merg.drop_duplicates(subset="tweet_id").itertuples():
        #print("row:", row)
        content = [
            eventidTopicidMap[row.eventID],
            "Q0",          
            np.int64(row.tweet_id),   
            getattr(row, 'Index'),
            str(priority_scorer[str(row.priority_y)[:3]]),
            #row.category_y,
            "['" + categoriser[round(row.postCategories_x_x,0)] + "']",
            "marksrun2"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")